In [41]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns

In [42]:
deaths_summed = pd.read_csv('total_deaths_perstate.csv',index_col=[0])
deaths_summed = deaths_summed[['State','Deaths']]

deaths_bydate = pd.read_csv('all_deaths_total.csv', index_col=[0]) 
deaths_bydate.columns = deaths_bydate.columns.str.capitalize()
deaths_bydate = deaths_bydate[['State','Deaths']]

macro =  pd.read_csv('macro_df.csv', index_col=[0])
macro = macro[['State','Percent Hispanic Or Latino','Percent White Not Hispanic Or Latino','Median Household Income']]

health = pd.read_csv('health.csv',index_col=[0])
health.rename(index={'State Name':'State'},inplace=True)

pop = pd.read_csv('state_pop_2021.csv', index_col=[0])
pop.columns = pop.columns.str.capitalize()

#print(deaths_summed.head())
#print(deaths_bydate.head())
#print(macro.head())
#print(health.head())
#print(pop.head())

In [43]:
#print(health.index)
health['State'] = health.index

In [44]:
#Create final table with all determined features 
from functools import reduce

features = reduce(lambda left, right:     
                     pd.merge(left , right,
                              on = ['State']),
                     [deaths_summed, health, macro, pop])

print(features.head())

        State  Deaths  Access to Care - AnnualRank  \
0     Alabama   20505                         45.0   
1      Alaska    1350                         20.0   
2     Arizona   31455                         42.0   
3    Arkansas   12347                         43.0   
4  California   96442                         18.0   

   Adverse Childhood ExperiencesRank  Air PollutionRank  \
0                               30.0               33.0   
1                               44.0               16.0   
2                               35.0               46.0   
3                               48.0               23.0   
4                                9.0               50.0   

   Air and Water Quality - AnnualRank  All Determinants - AnnualRank  \
0                                29.0                           45.0   
1                                43.0                           31.0   
2                                41.0                           32.0   
3                               

In [45]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(features.drop('State', axis=1))
scaled_features = scaler.transform(features.drop('State', axis=1))
scaled_data = pd.DataFrame(scaled_features, columns = features.drop('State', axis=1).columns)

In [46]:
#clean dataset so can run modeling 
scaled_data = scaled_data.replace([np.inf, -np.inf], np.nan)
scaled_data = scaled_data.dropna()
scaled_data = scaled_data.reset_index()
print(np.any(np.isnan(scaled_data)))
print(np.all(np.isfinite(scaled_data)))

scaled_data.dtypes

False
True


index                                     int64
Deaths                                  float64
Access to Care - AnnualRank             float64
Adverse Childhood ExperiencesRank       float64
Air PollutionRank                       float64
                                         ...   
Percent Hispanic Or Latino              float64
Percent White Not Hispanic Or Latino    float64
Median Household Income                 float64
Population                              float64
% population affected                   float64
Length: 243, dtype: object

In [47]:
scaled_data.to_csv('scaled_data.csv')

In [ ]:
# Does my data set have any categorical data, such as Gender or day of the week?
# Do my features have data values that range from 0 - 100 or 0-1 or both and more?